### Exercise 12.2

Change the architecture of your DNN using convolutional layers. Use `Conv2D`, `MaxPooling2D`, `Dropout`, but also do not forget `Flatten`, a standard `Dense` layer and `soft-max` in the end. I have merged step 2 and 3 in the following definition of `create_CNN()` that **<span style="color:red">you should complete</span>**:

In [3]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
seed = 0
np.random.seed(seed) # fix random seed
tf.random.set_seed(seed)

2024-06-10 18:49:16.081775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
from keras.datasets import mnist

# input image dimensions
img_rows, img_cols = 28, 28 # number of pixels 
# output
num_classes = 10 # 10 digits

# the data, split between train and test sets
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Consider an array of 5 labels out of a set of 3 classes {0, 1, 2}:
labels = np.array([0, 2, 1, 2, 0])
# `to_categorical` converts this into a matrix with as many columns as there are classes.
# The number of rows stays the same.
keras.utils.to_categorical(labels) # transform the array of labels into a matrix with as many columns as there are classes
# It's not binary, it's an encoding of the labels (categorical) for the cross-entropy loss function 

# reshape data, it could depend on Keras backend
X_train = X_train.reshape(X_train.shape[0], img_rows*img_cols) # reshape input image to a vector
X_test = X_test.reshape(X_test.shape[0], img_rows*img_cols)

# cast to floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# rescale data in interval [0,1] (normalize)
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices, e.g. for use with categorical_crossentropy
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)


#### Creating Convolutional Neural Nets with Keras

We have so far considered each MNIST data sample as a $(28\times 28,)$-long 1d vector. On the other hand, we do know that in every one of the hand-written digits there are *local* spatial correlations between the pixels, but also *translational invariance*, which we would like to take advantage of to improve the accuracy of our classification model. To this end, we first need to reshape the training and test input data as follows

In [ ]:

def create_CNN():
    # instantiate model
    model = Sequential()
    # add first convolutional layer with 10 filters (dimensionality of output space)
    model.add(Conv2D(10, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))
    #
    # ADD HERE SOME OTHER LAYERS AT YOUR WILL, FOR EXAMPLE SOME: Dropout, 2D pooling, 2D convolutional etc. ... 
    # remember to move towards a standard flat layer in the final part of your DNN,
    # and that we need a soft-max layer with num_classes=10 possible outputs
      # Add a second convolutional layer with 20 filters
    model.add(Conv2D(20, kernel_size=(5, 5), activation='relu'))
    
    # Add a max-pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add a dropout layer for regularization
    model.add(Dropout(0.25))

    # Add a third convolutional layer with 50 filters
    model.add(Conv2D(50, kernel_size=(3, 3), activation='relu'))

    # Add another max-pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Add another dropout layer
    model.add(Dropout(0.25))

    # Flatten the layers to transition from 2D to 1D
    model.add(Flatten())

    # Add a fully connected layer
    model.add(Dense(128, activation='relu'))

    # Add another dropout layer
    model.add(Dropout(0.5))

    # Add the final output layer with softmax activation
    model.add(Dense(num_classes, activation='softmax'))
    
    # compile the model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='SGD',
                  metrics=['acc'])
    return model

In [ ]:
# reshape data, depending on Keras backend
if keras.backend.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print()
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
Y_train shape: (60000, 10)

60000 train samples
10000 test samples


Train your DCNN and evaluate its performance proceeding exactly as before:

In [ ]:
# training parameters
batch_size = 32
epochs = 50

# create the deep conv net
model_CNN=create_CNN()

# train CNN
history = model_CNN.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test))

# evaliate model
score = model_CNN.evaluate(X_test, Y_test, verbose=1)

# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
1875/1875 [==============================] - 16s 8ms/step - loss: 0.8601 - acc: 0.7074 - val_loss: 0.1602 - val_acc: 0.9546
Epoch 2/50
1875/1875 [==============================] - 17s 9ms/step - loss: 0.2613 - acc: 0.9196 - val_loss: 0.1010 - val_acc: 0.9688
Epoch 3/50
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1973 - acc: 0.9402 - val_loss: 0.0763 - val_acc: 0.9764
Epoch 4/50
1875/1875 [==============================] - 16s 9ms/step - loss: 0.1653 - acc: 0.9495 - val_loss: 0.0641 - val_acc: 0.9796
Epoch 5/50
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1451 - acc: 0.9565 - val_loss: 0.0567 - val_acc: 0.9820
Epoch 6/50
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1297 - acc: 0.9611 - val_loss: 0.0510 - val_acc: 0.9840
Epoch 7/50
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1183 - acc: 0.9650 - val_loss: 0.0460 - val_acc: 0.9847
Epoch 8/50
1875/1875 [==============================] -

In [ ]:
# save model and history
# UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. 
# We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
model_CNN.save('DATA/models/CNN/model_CNN1.keras')
# print data to file
epochs = range(1, len(history.history['loss']) + 1)
data = {
    'epoch': epochs,
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss'],
    'acc': history.history['acc'],
    'val_acc': history.history['val_acc']
}

df = pd.DataFrame(data)
df.to_csv(f'DATA/DCNN.dat', index=False, sep = ' ')